In [12]:
import os
import numpy as np
import pandas as pd
import warnings
from tqdm import trange
from scipy.linalg import solve
from revised_cdcs.core.testAn import compute_test_tensor_G
from revised_cdcs.core.bnb_helper_anm import bnb_helper_anm
from revised_cdcs.test import generate_scenarios, get_scenario_description

In [2]:
def evaluate_test_performance(X: np.ndarray, Y: np.ndarray, reps: int=100, 
                              alpha: float=0.05, bs: int=400, norm: int=3):
    """
    Evaluates the power or type I error of the independence test on fixed (X, Y) data.
    
    Parameters
    ----------
    X: np.ndarray
        A 1D or 2D array of shape (n,) or (n, d1) representing variable X.
    Y: np.ndarray
        A 1D or 2D array of shape (n,) or (n, d2) representing variable Y.
    reps: int, optional
        Number of repetitions to simulate the test, by default 100.
    alpha: float, optional
        Significance level for hypothesis testing, by default 0.05.
    bs: int, optional
        number of bootstrap resamples, by default 400.
    norm: int, optional
        which norm to aggregate over test functions, by default 'inf'.
    
    Returns
    -------
    float
        Estimated type I error rate,
        computed as the proportion of times the null hypothesis is rejected.
    """
    # Ensure X and Y are 2D arrays
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    if Y.ndim == 1:
        Y = Y.reshape(-1, 1)

    n = X.shape[0]

    pvals = []
    for _ in trange(reps, desc="Running tests"):
        # Resample X and Y
        idx = np.random.choice(n, n, replace=True)
        X_resampled = X[idx]
        Y_resampled = Y[idx]
        G_resampled = compute_test_tensor_G(X_resampled)

        # Run the independence test
        pval = bnb_helper_anm(
            ancest=X_resampled, 
            children=Y_resampled, 
            G=G_resampled,
            withinAgg=norm, 
            aggType=norm, 
            bs=bs,
            bootstrap_method=2
        )
        pvals.append(pval)
    
    pvals = np.array(pvals)
    rate = np.mean(pvals < alpha)

    print(f"Rejection rate at alpha={alpha:.2f}: {rate:.3f}")
    
    return rate

In [16]:
n = 1000

scenarios = generate_scenarios()
print(f"Available scenarios: {len(scenarios)}")
for name in scenarios.keys():
    print(f"- {name}: {get_scenario_description(name)}")

Available scenarios: 55
- Independent GG 1: Independent Gaussian-Gaussian variables
- Independent GG 2: Independent Gaussian-Gaussian variables
- Independent GG 3: Independent Gaussian-Gaussian variables
- Independent GG 4: Independent Gaussian-Gaussian variables
- Independent GG 5: Independent Gaussian-Gaussian variables
- Independent GG 6: Independent Gaussian-Gaussian variables
- Independent GG 7: Independent Gaussian-Gaussian variables
- Independent GG 8: Independent Gaussian-Gaussian variables
- Independent GG 9: Independent Gaussian-Gaussian variables
- Independent GG 10: Independent Gaussian-Gaussian variables
- Independent GN 1: Independent Gaussian-NonGaussian variables
- Independent GN 2: Independent Gaussian-NonGaussian variables
- Independent GN 3: Independent Gaussian-NonGaussian variables
- Independent GN 4: Independent Gaussian-NonGaussian variables
- Independent GN 5: Independent Gaussian-NonGaussian variables
- Independent GN 6: Independent Gaussian-NonGaussian variabl

#### CASE 1: Independent Gaussian-Gaussian variables

In [17]:
scenario_types = {f"Independent GG {i}": "Independent (G-G)" for i in range(1, 11)} 

for name, gen_func in scenarios.items():
    if name.startswith("Independent GG"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Independent GG 1


Running tests: 100%|██████████| 100/100 [00:01<00:00, 51.67it/s]


Rejection rate at alpha=0.05: 0.220
Testing scenario: Independent GG 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 52.80it/s]


Rejection rate at alpha=0.05: 0.260
Testing scenario: Independent GG 3


...

Rejection rate at alpha=0.05: 0.210
Testing scenario: Independent GG 9


Running tests: 100%|██████████| 100/100 [00:01<00:00, 50.28it/s]


Rejection rate at alpha=0.05: 0.130
Testing scenario: Independent GG 10


Running tests: 100%|██████████| 100/100 [00:01<00:00, 51.55it/s]

Rejection rate at alpha=0.05: 0.230


#### CASE 2: Independent Gaussian-NonGaussian variables

In [18]:
scenario_types = {f"Independent GN {i}": "Independent (G-NG)" for i in range(1, 11)}

for name, gen_func in scenarios.items():
    if name.startswith("Independent GN"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Independent GN 1


Running tests: 100%|██████████| 100/100 [00:01<00:00, 52.66it/s]


Rejection rate at alpha=0.05: 0.060
Testing scenario: Independent GN 2


Running tests: 100%|██████████| 100/100 [00:02<00:00, 49.82it/s]


Rejection rate at alpha=0.05: 0.150
Testing scenario: Independent GN 3


...

Rejection rate at alpha=0.05: 0.280
Testing scenario: Independent GN 9


Running tests: 100%|██████████| 100/100 [00:01<00:00, 50.69it/s]


Rejection rate at alpha=0.05: 0.280
Testing scenario: Independent GN 10


Running tests: 100%|██████████| 100/100 [00:01<00:00, 51.85it/s]

Rejection rate at alpha=0.05: 0.400


#### CASE 3: Independent NonGaussian-NonGaussian variables

In [6]:
scenario_types = {f"Independent NN {i}": "Independent (NG-NG)" for i in range(1, 11)} 

for name, gen_func in scenarios.items():
    if name.startswith("Independent NN"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Independent NN 1


Running tests: 100%|██████████| 100/100 [00:01<00:00, 55.59it/s]


Rejection rate at alpha=0.05: 0.160
Testing scenario: Independent NN 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.15it/s]


Rejection rate at alpha=0.05: 0.220
Testing scenario: Independent NN 3


...

Rejection rate at alpha=0.05: 0.330
Testing scenario: Independent NN 9


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.31it/s]


Rejection rate at alpha=0.05: 0.270
Testing scenario: Independent NN 10


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.57it/s]

Rejection rate at alpha=0.05: 0.300


#### CASE 4: Weak nonlinear dependence (hard to detect)

In [7]:
scenario_types = {f"Subtle Weak {i}": "Subtle Dependent" for i in range(1, 6)}

for name, gen_func in scenarios.items():
    if name.startswith("Subtle Weak"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Subtle Weak 1


Running tests: 100%|██████████| 100/100 [00:01<00:00, 52.32it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Weak 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.10it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Weak 3


...

Rejection rate at alpha=0.05: 0.260
Testing scenario: Subtle Weak 4


Running tests: 100%|██████████| 100/100 [00:01<00:00, 51.65it/s]


Rejection rate at alpha=0.05: 0.110
Testing scenario: Subtle Weak 5


Running tests: 100%|██████████| 100/100 [00:01<00:00, 54.84it/s]

Rejection rate at alpha=0.05: 0.950


#### CASE 5: Conditional dependence through latent variable

In [8]:
scenario_types = {f"Subtle Cond {i}": "Subtle Dependent" for i in range(1, 6)}

for name, gen_func in scenarios.items():
    if name.startswith("Subtle Cond"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Subtle Cond 1


Running tests: 100%|██████████| 100/100 [00:01<00:00, 51.98it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Cond 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.22it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Cond 3


...

Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Cond 4


Running tests: 100%|██████████| 100/100 [00:01<00:00, 50.83it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Cond 5


Running tests: 100%|██████████| 100/100 [00:01<00:00, 54.48it/s]

Rejection rate at alpha=0.05: 1.000


#### CASE 6: Higher-order interaction dependence

In [9]:
scenario_types = {f"Subtle Higher {i}": "Subtle Dependent" for i in range(1, 6)}

for name, gen_func in scenarios.items():
    if name.startswith("Subtle Higher"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Subtle Higher 1


Running tests: 100%|██████████| 100/100 [00:01<00:00, 54.71it/s]


Rejection rate at alpha=0.05: 0.390
Testing scenario: Subtle Higher 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 54.24it/s]


Rejection rate at alpha=0.05: 0.490
Testing scenario: Subtle Higher 3


...

Rejection rate at alpha=0.05: 0.370
Testing scenario: Subtle Higher 4


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.83it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Subtle Higher 5


Running tests: 100%|██████████| 100/100 [00:01<00:00, 50.76it/s]

Rejection rate at alpha=0.05: 1.000


#### CASE 7: Strong nonlinear dependence (easy to detect)

In [10]:
scenario_types = {f"Dependent Strong {i}": "Clearly Dependent" for i in range(1, 6)}

for name, gen_func in scenarios.items():
    if name.startswith("Dependent Strong"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Dependent Strong 1


Running tests: 100%|██████████| 100/100 [00:02<00:00, 39.73it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Dependent Strong 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 54.94it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Dependent Strong 3


...

Rejection rate at alpha=0.05: 1.000
Testing scenario: Dependent Strong 4


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.10it/s]


Rejection rate at alpha=0.05: 0.480
Testing scenario: Dependent Strong 5


Running tests: 100%|██████████| 100/100 [00:02<00:00, 47.99it/s]

Rejection rate at alpha=0.05: 1.000


#### CASE 8: Dependence between mixed variable types

In [11]:
scenario_types = {f"Dependent Mixed {i}": "Clearly Dependent" for i in range(1, 6)}

for name, gen_func in scenarios.items():
    if name.startswith("Dependent Mixed"):
        print(f"Testing scenario: {name}")
        X, Y = gen_func(n)
        rejection_rate = evaluate_test_performance(X, Y, reps=100, alpha=0.05, bs=400, norm=3)

Testing scenario: Dependent Mixed 1


Running tests: 100%|██████████| 100/100 [00:02<00:00, 43.68it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Dependent Mixed 2


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.19it/s]


Rejection rate at alpha=0.05: 1.000
Testing scenario: Dependent Mixed 3


...

Rejection rate at alpha=0.05: 1.000
Testing scenario: Dependent Mixed 4


Running tests: 100%|██████████| 100/100 [00:01<00:00, 53.51it/s]


Rejection rate at alpha=0.05: 0.080
Testing scenario: Dependent Mixed 5


Running tests: 100%|██████████| 100/100 [00:01<00:00, 54.53it/s]

Rejection rate at alpha=0.05: 0.100
